## Modularity
### H3: Random Hypothesis
* Randomly wired networks lack an inherent community structure.
* By comparing the link density of a community with the link density obtained for the same group of nodes for a randomly rewired network, we could decide if the original community corresponds to a dense subgraph, or its connectivity pattern emerged by chance.


### Modularity
*The "quality" of systematic deviations from a random configuration of a graph.*
* Allows us to rate communities and compare them

Consider a graph, G:
* $N$ nodes, $L$ links
* $n_c$ communities, each having $N_c$ nodes and $L_c$ links ($c = 1..n_c$)
* If $L_c > L_{expect}(C_c)$ then C_c is likely a community
* Measure difference between actual wiring diagram ($A_{ij}$) and the expected number of links between $i$ and $j$ in a random graph with wiring probability $p_{ij}$.

$M_c = \frac{1}{2L} \sum_{i,j \in C_c}{(A_{ij} - p_{ij})} = 
\frac{L_C}{L} - \left(\frac{k_c}{2L}\right)^2 $, 
where $p_{ij} = \frac{k_i k_j}{2L}$

* If $M_c > 0$ then the subgraph $C_c$ has more links than expected and represents a potential community.
* If $M_c = 0$ then the connectivity is what can be expected from a random graph, and nothing final can be concluded.
* If $M_c = 0$ then $C_c$ is not a community.

Modularity of a partition which breaks a graph into $n_c$ communities:
$M = \sum_{c=1}^{c_n}M_c$





* Explain the concept of modularity in your own words.
    * A measure that allows us to compare the quality of communities in a quantitative manner.

Consider the undirected version of the graph for the 115th house of representatives.
* Compute the modularity when you partition nodes based on their party. Modularity is described in the Network Science book, section 9.4). * * Use equation 9.12 in the book to calculate the modularity M of the parties-partitioning. Are the parties good community?


In [4]:
# Load politician class and aux functions
%run ../week5/aux.py

In [5]:
url_h115 = 'https://raw.githubusercontent.com/suneman/socialgraphs2018/master/files/data_US_congress/H115.csv'
df = pd.read_csv(url_h115)
page_names = df.WikiPageName

In [7]:
congress_graph = nx.Graph()

for index, row in df.iterrows():
    # Create politician node
    p = Politician(row)
    congress_graph.add_node(p)
    
    # Get links
    wikiPageName = row[0]
    articleLinks = get_article_links(wikiPageName, "", "/congress115/")
    for link in articleLinks:
        if link in wiki_page_names:
            linkIndex = wiki_page_names.index(link)
            linkRow = df.iloc[linkIndex]
            q = Politician(linkRow)
            congress_graph.add_edge(p, q)

FileNotFoundError: [Errno 2] No such file or directory: '/congress115/John_Conyers.txt'

* Repeat the exercise above by considering states instead of parties. Are the states good communities?


* Would you expect these results in light of what we have found in the previous exercises?